# Overlay

Overlay-analyser er GIS-operasjoner der to eller flere vektorlag er
kombinert for å produsere nye geometrier. Typiske overlay-operasjoner inkluderer *union*,
*intersection*, og *difference* - navngitt etter resultatet av kombinasjonen av to lag.


![Fire paneler som viser union, intersection, symmetric difference og difference av to geometrier.](imgs/overlay_operations.png)


Romlig overlay med to inputvektorlag (rektangel, sirkel). Det resulterende vektorlaget vises i grønt. *Kilde: [QGIS dokumentasjon](https://docs.qgis.org/latest/en/docs/gentle_gis_introduction/vector_spatial_analysis_buffers.html#figure-overlay-operations)*
:::


I denne notebooken vil vi utføre en overlay-analyse for å velge de polygon
cellene i et nett datasett som ligger innenfor bygrensene i Helsinki. For denne
øvelsen, bruker vi to inngangs datasett: et nett av statistiske polygoner med
reisetiden til Helsingfors jernbanestasjon, som dekker hele storbyområdet (`helsinki_region_travel_times_to_railway_station.gpkg`) og et polygon
datasett (med en funksjon) av området kommunen Helsingfors dekker
(`helsinki_municipality.gpkg`). Begge filene er i logisk navngitte undermapper
av `DATA_DIRECTORY`.

In [ ]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas

nett = geopandas.read_file(
    DATA_DIRECTORY
    / "helsinki_region_travel_times_to_railway_station"
    / "helsinki_region_travel_times_to_railway_station.gpkg"
)

helsinki = geopandas.read_file(
    DATA_DIRECTORY / "helsinki_municipality" / "helsinki_municipality.gpkg"
)

La oss gjøre en rask overleggsvisualisering av de to lagene:

In [ ]:
# Plott lagene
ax = nett.plot(facecolor="gray")
helsinki.plot(ax=ax, facecolor="None", edgecolor="blue")

Her er det grå området Reisetid Matrise - et datasett som inneholder 13231
rutenett kvadrater (13231 rader med data) som dekker Helsingfors-regionen, og det blå området representerer kommunen Helsingfors. Målet vårt er å utføre en overleggsanalyse og velge geometriene fra rutenett polygonlaget som krysser
med Helsingfors kommune polygon.

Når du utfører overleggsanalyse, er det viktig å først sjekke at CRS
av lagene matcher. Overleggsvisualiseringen indikerer at alt burde være
ok (lagene plottes pent oppå hverandre). Men, la oss
likevel sjekke om crs matcher ved hjelp av Python:

In [ ]:
# Sjekk crs av kommunen polygon
print(helsinki.crs)

In [ ]:
# Sørg for at CRS matcher, hvis ikke løft en AssertionError
assert helsinki.crs == nett.crs, "CRS skiller seg mellom lagene!"

Faktisk, det gjør de. Vi er nå klare til å utføre en overleggsanalyse mellom disse lagene. 

Vi vil lage et nytt lag basert på rutenett polygoner som `intersect` med vår
Helsingfors lag. Vi kan bruke en metode `overlay()` av en `GeoDataFrame` for å utføre
overleggsanalysen som tar som en input 1) andre GeoDataFrame, og 2)
parameter `how` som kan brukes til å kontrollere hvordan overleggsanalysen er
utført (mulige verdier er `'intersection'`, `'union'`,
`'symmetric_difference'`, `'difference'`, og `'identity'`):

In [ ]:
intersection = nett.overlay(helsinki, how="intersection")

La oss plotte dataene våre og se hva vi har:

In [ ]:
intersection.plot(color="b")

Som et resultat har vi nå bare de rutenett cellene som krysser med Helsingfors
grenser. Hvis du ser nøye etter, kan du også observere at **rutenett cellene er
klippet basert på grensen.**

- Hva med dataattributter? La oss se hva vi har:

In [ ]:
intersection.head()

Som vi kan se, på grunn av overleggsanalysen, inneholder datasettet attributtene
fra begge input lagene.

La oss lagre resultatrutenettet vårt som en GeoPackage.

In [ ]:
intersection.to_file(
    DATA_DIRECTORY / "intersection.gpkg",
    layer="travel_time_matrix_helsinki_region"
)

Det er mange flere eksempler på forskjellige typer overleggsanalyse i
[Geopandas dokumentasjon](http://geopandas.org/set_operations.html) hvor du
kan gå og lære mer.